Just a quick function to present out NSE findings per model per basin

In [2]:
import pandas as pd
import plotly.graph_objects as go

In [3]:
def create_nse_table(basins, models, nse_values):
    """
    Create a table using Plotly where rows are basins, columns are models, and
    the highest NSE for each row is highlighted and bolded.

    Parameters:
    - basins: List of basin names.
    - models: List of model names.
    - nse_values: 2D list of NSE values, where each row corresponds to a basin and each column to a model.

    Returns:
    - A Plotly table figure.
    """
    # Create a DataFrame for easier manipulation
    df = pd.DataFrame(nse_values, index=basins, columns=models)

    # Highlight the maximum value in each row
    formatted_values = []
    for basin in df.index:
        row = []
        max_value = df.loc[basin].max()
        for value in df.loc[basin]:
            if value == max_value:
                row.append(f"<b>{value:.3f}</b>")
            else:
                row.append(f"{value:.3f}")
        formatted_values.append(row)

    # Create the table figure
    fig = go.Figure(
            data=[
                go.Table(
                    header=dict(
                        values=["<b>Basin</b>"] + [f"<b>{model}</b>" for model in models],
                        fill_color="grey",  # Professional blue
                        font=dict(color="white", size=14),  # White text for header
                        align="center",
                    ),
                    cells=dict(
                        values=[[basin for basin in basins]] + list(map(list, zip(*formatted_values))),
                        fill=dict(
                            color=[
                                ["#F9F9F9" if i % 2 == 0 else "#FFFFFF" for i in range(len(basins))],  # Alternating row colors
                                ["#F9F9F9" if i % 2 == 0 else "#FFFFFF" for i in range(len(basins))],
                                ["#F9F9F9" if i % 2 == 0 else "#FFFFFF" for i in range(len(basins))],
                                ["#F9F9F9" if i % 2 == 0 else "#FFFFFF" for i in range(len(basins))],
                            ]
                        ),
                        align="center",
                        font=dict(color="#333333", size=12),  # Dark gray text for cells
                    ),
                )
            ]
        )


    return fig

Once all the runs have been done get the NSE outputted by results and update nse_values with real values:

In [4]:
# Example usage: input the actual NSE Values
basins = ["Calpella", "Hopland", "Guerneville", "Warm Springs"]
models = ["HMS NSE", "LSTM NSE", "Physics Informed LSTM NSE"]
nse_values = [
    [0.70, 0.79, 0.82], #calpella --> updated with real numbers from calpella model notebook (includes all physics columns)
    [0.80, 0.72, 0.75], #hopland --> updated with real numbers from hopland model notebook (includes all physics columns)
    [0.85, 0.65, 0.74], #guerneville--> updated with real numbers from guerneville model notebook (includes full physics)
    [0.73, 0.85, 0.82], #warm springs --> updated with real numbers from warm_springs model notebook (includes all physics columns)
]

fig = create_nse_table(basins, models, nse_values)
fig.show()

In [67]:
def create_metrics_table(dataframe, title):
    """
    Create a Plotly table with alternating row colors and a specified title.

    Parameters:
    - dataframe: A Pandas DataFrame containing the metrics data.
        The first column should contain metric names (rows), and
        the remaining columns should contain model values.
    - title: A string for the table title.

    Returns:
    - A Plotly Figure object.
    """
    # Clean the data: Rename and drop unnecessary rows/columns
    data_cleaned = dataframe.dropna().rename(columns={'Unnamed: 0': 'Metric'})

    # Extract table values
    metrics = data_cleaned['Metric'].tolist()  # Metrics as rows
    models = list(data_cleaned.columns[1:])  # Columns are model names
    values = [data_cleaned[col].tolist() for col in models]  # Model values per metric

    # Create alternating row colors
    row_colors = ["#F9F9F9" if i % 2 == 0 else "#FFFFFF" for i in range(len(metrics))]

    # Create Plotly table
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['<b>Metric</b>'] + [f"<b>{model}</b>" for model in models],
            fill_color="grey",  # Header background color
            font=dict(size=12, color='white'),  # Header text color
            align='center'
        ),
        cells=dict(
            values=[metrics] + values,
            fill_color=[row_colors * len(models)],  # Alternating row colors
            align='center',
            font=dict(size=11, color='black'),
            format=[None] + [".4f"] * len(models)  # Adjust number format
        )
    )])

    # Customize layout
    fig.update_layout(
        title=title,
        title_x=0.5,  # Center the title
        width=1200,
        height=600
    )

    return fig

In [68]:
calpella_metrics_df = pd.read_csv("calpella_all/calpella_metrics.csv")
calpella_metrics_df

,Unnamed: 0,HMS,LSTM,Physics_Informed_LSTM
0,NSE,0.699186,0.774333,0.724424
1,MSE,76611.326933,57472.847658,28870.322390
2,RMSE,276.787512,239.734953,169.912690
3,KGE,0.746424,0.675719,0.808272
4,Alpha-NSE,1.224704,0.712526,0.910376
5,Beta-KGE,1.058859,0.885681,1.087094
6,Beta-NSE,0.034577,-0.067157,0.052941
7,Pearson-r,0.898291,0.902799,0.854598
8,FHV,25.593778,-25.316821,-3.790094
9,FMS,0.594171,-32.859758,-41.091226


In [71]:
calpella_table = create_metrics_table(calpella_metrics_df, "Metrics Table for Calpella")
calpella_table.show()

In [73]:
warm_springs_metrics_df = pd.read_csv("warm_springs_all/warm_springs_metrics.csv")
warm_springs_metrics_df

,Unnamed: 0,HMS,LSTM,Physics_Informed_LSTM
0,NSE,0.732378,0.850322,0.762767
1,MSE,143219.109139,80100.972954,126956.311357
2,RMSE,378.443006,283.021153,356.309292
3,KGE,0.791193,0.875194,0.739197
4,Alpha-NSE,1.179502,0.939637,0.797347
5,Beta-KGE,1.037973,1.077303,0.889880
6,Beta-NSE,0.015443,0.031438,-0.044784
7,Pearson-r,0.900313,0.922818,0.878247
8,FHV,24.454837,-1.038285,-23.831772
9,FMS,25.355981,-11.638515,73.623212


In [76]:
warm_springs_table = create_metrics_table(warm_springs_metrics_df, "Metrics Table for Warm Springs")
warm_springs_table.show()

In [77]:
guerneville_metrics_df = pd.read_csv("guerneville_all/guerneville_metrics.csv")
guerneville_metrics_df

,Unnamed: 0,HMS,LSTM,Physics_Informed_LSTM
0,NSE,8.499148e-01,6.494876e-01,7.409967e-01
1,MSE,4.623214e+06,1.079716e+07,7.978319e+06
2,RMSE,2.150166e+03,3.285903e+03,2.824592e+03
3,KGE,8.430314e-01,6.530352e-01,7.235612e-01
4,Alpha-NSE,1.026757e+00,7.137961e-01,7.665495e-01
5,Beta-KGE,1.137092e+00,1.056343e+00,1.064927e+00
6,Beta-NSE,4.800380e-02,1.972874e-02,2.273455e-02
7,Pearson-r,9.283838e-01,8.121239e-01,8.669445e-01
8,FHV,2.575216e+00,-3.882094e+01,-3.101388e+01
9,FMS,-2.409063e+01,1.180181e+02,3.235107e+01


In [78]:
guerneville_table = create_metrics_table(guerneville_metrics_df, "Metrics Table for Guerneville")
guerneville_table.show()

In [61]:
hopland_metrics_df = pd.read_csv("hopland_all/hopland_metrics.csv")
hopland_metrics_df

,Unnamed: 0,HMS,LSTM,Physics_Informed_LSTM
0,NSE,0.795537,0.723541,0.753028
1,MSE,404943.189982,547532.777360,489132.717098
2,RMSE,636.351467,739.954578,699.380238
3,KGE,0.728365,0.712020,0.759963
4,Alpha-NSE,1.186275,0.777365,0.841604
5,Beta-KGE,1.185323,1.111813,1.124909
6,Beta-NSE,0.079849,0.048176,0.053819
7,Pearson-r,0.931134,0.855556,0.869899
8,FHV,23.945896,-20.731369,-14.677648
9,FMS,32.032020,18.998075,46.100325


In [79]:
hopland_table = create_metrics_table(hopland_metrics_df, "Metrics Table for Hopland")
hopland_table.show()

In [55]:
# Add basin names to columns to create hierarchical column structure
calpella_metrics_df.columns = pd.MultiIndex.from_product([["Calpella"], calpella_metrics_df.columns])
warm_springs_metrics_df.columns = pd.MultiIndex.from_product([["Warm Springs"], warm_springs_metrics_df.columns])
guerneville_metrics_df.columns = pd.MultiIndex.from_product([["Guerneville"], guerneville_metrics_df.columns])
hopland_metrics_df.columns = pd.MultiIndex.from_product([["Hopland"], hopland_metrics_df.columns])

# Combine all the metrics into one DataFrame
combined_metrics_df = pd.concat(
    [hopland_metrics_df, guerneville_metrics_df, warm_springs_metrics_df, calpella_metrics_df], axis=1
)
combined_metrics_df
output_csv_path = "all_metrics.csv"
combined_metrics_df.to_csv(output_csv_path)

In [ ]:

styled_table = combined_metrics_df.style.set_table_styles(
    [{
        'selector': 'thead th',
        'props': [('background-color', '#D3D3D3'), ('color', 'black'), ('font-weight', 'bold')]
    }, {
        'selector': 'tbody td',
        'props': [('text-align', 'center')]
    }]
).set_caption("Metrics Table for All Basins and Models")

styled_table

In [57]:
# Load the data
# Restructure data: Ensure proper header alignment and remove unnecessary rows
data_cleaned = combined_metrics_df.dropna()  # Drop rows with NaN values
data_cleaned = data_cleaned.rename(columns={'Unnamed: 0': 'Metric'})  # Rename the first column

# Create a Plotly table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=data_cleaned.columns,
        fill_color='paleturquoise',
        align='left',
        font=dict(size=12, color='black')
    ),
    cells=dict(
        values=[data_cleaned[col] for col in data_cleaned.columns],
        fill_color='lavender',
        align='left',
        font=dict(size=11, color='black')
    )
)])

# Customize layout
fig.update_layout(
    title_text="Metrics Table",
    title_x=0.5,  # Center the title
    width=1200,
    height=600
)

# Show the table
fig.show()